# Lesson 4 - Dataset Pools

Make imports

In [ ]:
from ansys.mapdl.core import MapdlPool

Create several onstances of mapdl

In [ ]:
# Initialize 3 instances of mapdl running on 8 processors in SMP mode
pool = MapdlPool(n_instances=3, nproc=8, additional_switches="-smp")

In [ ]:
# Set mapdl model as a function
def model(mapdl, load_x_inches, load_y_inches):
    """
    Get mapdl model as a function to interact with pool
    """
    # Define position of sensors
    # Define area parameters
    length_inches = 4.5
    height_inches = 2

    # Define positions as a dictionary
    sensors = {
        "x1": 1,
        "y1": 0.5,
        "x2": 2.25,
        "y2": 0.5,
        "x3": 3.5,
        "y3": 0.5,
        "x4": 1,
        "y4": 1.5,
        "x5": 2.25,
        "y5": 1.5,
        "x6": 3.5,
        "y6": 1.5,
        "x7": 0.5,
        "y7": 1,
        "x8": 4,
        "y8": 1,
        "load_x": load_x_inches,
        "load_y": load_y_inches,
    }

    # Set sensors data as empty dictionary
    sensors_data = {}

    # Create rectangular area
    # Clear model
    mapdl.clear()

    # Enter pre-processor
    mapdl.prep7()

    # Create a rectangular area or block volume by corner points
    mapdl.blc4(0, 0, length_inches, height_inches, 0)

    # Delete un-meshed areas
    mapdl.adele("all")

    # Generate a fillet line between two intersecting lines
    mapdl.lfillt(1, 2, 0.2)
    mapdl.lfillt(2, 3, 0.2)
    mapdl.lfillt(3, 4, 0.2)
    mapdl.lfillt(4, 1, 0.2)

    # Compress the numbering of defined items
    mapdl.numcmp("area")

    # Generate an area bounded by previously defined lines
    mapdl.al("all")

    # Define hard points by sensor coordinates
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        mapdl.hptcreate(
            type_="area",
            entity=1,
            nhp="",
            label="coord",
            val1=sensors["x" + str(points)],
            val2=sensors["y" + str(points)],
            val3=0,
        )

    mapdl.hptcreate("area", 1, "", "coord", sensors["load_x"], sensors["load_y"], 0)

    # Merges coincident or equivalently defined items
    mapdl.nummrg("kp")

    # Set material properties
    mapdl.mp("ex", 1, 1e7)

    # Set element type
    mapdl.et(1, 181)

    # Set section type (shell representation)
    mapdl.sectype(1, "shell")

    # Set shell thickness
    mapdl.secdata(0.1, 1)

    # Specify the element shape as 2D
    mapdl.mshape(1, "2d")

    # Specify the element size to be meshed onto areas
    mapdl.smrtsize(3)

    # Generate nodes and area elements within areas
    mapdl.amesh("all")

    # Loop over nodes
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )

        # Refine the mesh around specified nodes
        mapdl.nrefine(node, level=2, depth=1)

    mapdl.nsel("none")
    mapdl.cm("sensors", "node")
    mapdl.allsel()

    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )
        mapdl.cmsel("s", "sensors")
        mapdl.nsel("a", "node", "", node)
        mapdl.cm("sensors", "node")
        mapdl.allsel()

    mapdl.finish()
    mapdl.slashsolu()
    mapdl.dl("all", "", "all", 0)

    force_node = mapdl.queries.node(sensors["load_x"], sensors["load_y"], 0)
    mapdl.f(force_node, "fz", 1)

    mapdl.nlgeom("on")
    solve = mapdl.solve()
    mapdl.finish()

    mapdl.post1()
    mapdl.set("last")

    mapdl.allsel()

    result = {}
    result["Load_X"] = sensors["load_x"]
    result["Load_Y"] = sensors["load_y"]
    for points in range(1, (int((len(sensors) - 2) / 2)) + 1):
        node = mapdl.queries.node(
            sensors["x" + str(points)], sensors["y" + str(points)], 0
        )
        strain_x = mapdl.get_value("node", node, "epto", "x")
        strain_y = mapdl.get_value("node", node, "epto", "y")
        result["Strn_X_" + str(points)] = strain_x
        result["Strn_Y_" + str(points)] = strain_y

    return result

In [ ]:
inputs = []
with open("pointset.csv", newline="") as csvfile:
    point_input = csv.reader(csvfile, delimiter=",", quotechar="|")
    for point in point_input:
        inputs += [(float(point[0]), float(point[1]))]
header = {
    "Load_X": [],
    "Load_Y": [],
    "Strn_X_1": [],
    "Strn_Y_1": [],
    "Strn_X_2": [],
    "Strn_Y_2": [],
    "Strn_X_3": [],
    "Strn_Y_3": [],
    "Strn_X_4": [],
    "Strn_Y_4": [],
    "Strn_X_5": [],
    "Strn_Y_5": [],
    "Strn_X_6": [],
    "Strn_Y_6": [],
    "Strn_X_7": [],
    "Strn_Y_7": [],
    "Strn_X_8": [],
    "Strn_Y_8": [],
}

data_1 = pd.DataFrame(header)
output = pool.map(model, inputs, progress_bar=True, wait=True)
data_2 = pd.DataFrame(output)
data_1 = pd.concat([data_1, data_2], ignore_index=True)

In [ ]:
data_1.to_pickle("ResultDataAll.pkl")
pool.exit()